In [10]:
from mytools import *

import pandas as pd
import numpy as np
import re
import json 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression

print("All modules imported.")

All modules imported.


In [4]:
traindf = load_csv("train")
traindf.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
0,marvelous_pirate,Benjamin Henry,False,Henry Selick’s first movie since 2009’s Corali...,POSITIVE
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,With a cast that reads like the Vogue Oscar pa...,NEGATIVE
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,Creed II does not give us anything but another...,POSITIVE
3,lara_croft_glimmer,Ryan Barrett,False,"I know what you're thinking, but this is no Li...",POSITIVE
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,Director Fernando Meirelles tells the story wi...,POSITIVE


In [5]:
train_features = traindf["reviewText"]
train_labels = traindf["sentiment"]

train_features.shape, train_labels.shape

((162758,), (162758,))

In [7]:
train_features.fillna(" ", inplace=True)

In [24]:
def make_vocab_file(df, stopwords=None, max_features=100, min_df=1, max_df=1.0):
    vocab = get_tfidf_vocab(df, stop_words=stopwords, max_features=max_features, min_df=min_df, max_df=max_df)
    filename = f"vocab/vocab_{stopwords}_{max_features}_{min_df}_{max_df}.json"
    with open(filename, "w") as f:
        json.dump(vocab, f)
        print("Job done!")

In [20]:
make_vocab_file(traindf, stopwords="english", max_features=100)

Job done!


In [21]:
make_vocab_file(traindf, stopwords=None, max_features=100)

Job done!


In [25]:
make_vocab_file(traindf, stopwords="english", max_features=100, max_df=0.3)

Job done!
